In [22]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
from advanced_ta import LorentzianClassification
from ta.volume import money_flow_index as MFI

In [23]:
# Define the stock symbol and date range
symbol = "RAILTEL.NS"
start_date = "2021-01-01"
end_date = "2024-08-05"

# Fetch the data
df = yf.download(symbol, start=start_date, end=end_date)
df = df.reset_index()
df = df.rename(columns={'Date':'date', 'Open':'open', 'Close':'close', 'High':'high', 'Low':'low', 'Adj Close':'adj close', 'Volume':'volume'}, inplace=False)
df

[*********************100%%**********************]  1 of 1 completed


,date,open,high,low,close,adj close,volume
0,2021-02-26,109.000000,127.849998,101.000000,121.349998,113.996506,128128642
1,2021-03-01,128.850006,145.600006,127.400002,145.600006,136.777039,44411780
2,2021-03-02,159.000000,174.699997,155.899994,174.699997,164.113617,47816924
3,2021-03-03,182.899994,189.699997,160.300003,165.550003,155.518112,44140719
4,2021-03-04,159.000000,169.550003,155.000000,162.449997,152.605957,16531447
...,...,...,...,...,...,...,...
841,2024-07-29,489.500000,516.000000,488.350006,509.700012,509.700012,13323161
842,2024-07-30,510.049988,518.900024,503.000000,510.750000,510.750000,6416199
843,2024-07-31,512.750000,513.200012,498.149994,500.100006,500.100006,2770724
844,2024-08-01,503.350006,505.500000,494.000000,496.399994,496.399994,2935815


In [24]:
lc = LorentzianClassification(df)
lc.df

,date,open,high,low,close,adj close,volume,isEmaUptrend,isEmaDowntrend,isSmaUptrend,...,barsHeld,isEarlySignalFlip,isLastSignalBuy,isLastSignalSell,isNewBuySignal,isNewSellSignal,startLongTrade,startShortTrade,endLongTrade,endShortTrade
0,2021-02-26,109.000000,127.849998,101.000000,121.349998,113.996506,128128642,True,True,True,...,1,False,False,False,False,False,NaN,NaN,NaN,NaN
1,2021-03-01,128.850006,145.600006,127.400002,145.600006,136.777039,44411780,True,True,True,...,2,False,False,False,False,False,NaN,NaN,NaN,NaN
2,2021-03-02,159.000000,174.699997,155.899994,174.699997,164.113617,47816924,True,True,True,...,3,False,False,False,False,False,NaN,NaN,NaN,NaN
3,2021-03-03,182.899994,189.699997,160.300003,165.550003,155.518112,44140719,True,True,True,...,4,False,False,False,False,False,NaN,NaN,NaN,NaN
4,2021-03-04,159.000000,169.550003,155.000000,162.449997,152.605957,16531447,True,True,True,...,5,False,False,False,False,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,2024-07-29,489.500000,516.000000,488.350006,509.700012,509.700012,13323161,True,True,True,...,28,False,True,False,False,False,NaN,NaN,NaN,NaN
842,2024-07-30,510.049988,518.900024,503.000000,510.750000,510.750000,6416199,True,True,True,...,29,False,True,False,False,False,NaN,NaN,NaN,NaN
843,2024-07-31,512.750000,513.200012,498.149994,500.100006,500.100006,2770724,True,True,True,...,30,False,True,False,False,False,NaN,NaN,NaN,NaN
844,2024-08-01,503.350006,505.500000,494.000000,496.399994,496.399994,2935815,True,True,True,...,31,False,True,False,False,False,NaN,NaN,NaN,NaN


In [25]:
lc = LorentzianClassification(
    df,
    features=[
        LorentzianClassification.Feature("RSI", 14, 2),  # f1
        LorentzianClassification.Feature("WT", 10, 11),  # f2
        LorentzianClassification.Feature("CCI", 20, 2),  # f3
        LorentzianClassification.Feature("ADX", 20, 2),  # f4
        LorentzianClassification.Feature("RSI", 9, 2),   # f5
        MFI(df['open'], df['high'], df['low'], df['close'], df['volume'], 14) #f6
    ],
    settings=LorentzianClassification.Settings(
        source='close',
        neighborsCount=8,
        maxBarsBack=2000,
        useDynamicExits=False
    ),
    filterSettings=LorentzianClassification.FilterSettings(
        useVolatilityFilter=True,
        useRegimeFilter=True,
        useAdxFilter=False,
        regimeThreshold=-0.1,
        adxThreshold=20,
        kernelFilter = LorentzianClassification.KernelFilter(
            useKernelSmoothing = False,
            lookbackWindow = 8,
            relativeWeight = 8.0,
            regressionLevel = 25,
            crossoverLag = 2
        )
    ))
lc.dump('output/result.csv')
lc.plot('output/result.jpg')

ValueError: window must be an integer 0 or greater